In [2]:
from langchain_ollama import OllamaLLM
from smolagents import LiteLLMModel

langchain_model = OllamaLLM(model="llama3.2")
smolagents_model = LiteLLMModel(
    model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)

In [3]:
dict_prompt = {
    'style': 'Pop',
    'user_prompt': 'A song about love and resilience',
    'tone': 'Sad'
}

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

def create_chain(parser_class, template, variables):
    parser = JsonOutputParser(pydantic_object=parser_class)
    prompt = PromptTemplate(
        template=template,
        input_variables=variables
    )

    chain = prompt | langchain_model | parser

    return chain

In [ ]:
class Lyrics(BaseModel):
    title: str = Field(description="Title of the song")
    intro: str = Field(description="Intro to the song")
    verse_one: str = Field(description="Verse one of the song")
    pre_chorus: str = Field(description="Pre-chorus of the song")
    chorus: str = Field(description="Chorus of the song")
    verse_two: str = Field(description="Verse two of the song")
    outro: str = Field(description="Outro of the song")


lyrics_template = '''You are a talented songwriter.
Consider a song with the following requisites:
- Style: {style}
- Prompt: {user_prompt}
- Tone: {tone}
Create the lyrics for this song, with the following sections: intro, verse one, pre-chorus, chorus, verse two and outro.
Ideally, create a strophe with at least 2 verses for each section. Finally, give it a title.
Answer in the following JSON format:
    - "title": title of the song
    - "intro": intro to the song
    - "verse_one": Verse one of the song
    - "pre_chorus": Pre-chorus of the song
    - "chorus": Chorus of the song
    - "verse_two": Verse two of the song
    - "outro": Outro of the song
'''

chain = create_chain(
    parser_class=Lyrics,
    template=lyrics_template,
    variables=['style', 'user_prompt', 'tone']
)

lyrics = chain.invoke(dict_prompt)

In [9]:
lyrics = {"title": "Fading Away",
 "intro": ["In the silence, I hear your voice",
  "A whispered reminder of our choice",
  "To let go, to move on, to lose",
  "The love we had, the memories we chose"],
 "verse_one": ["We said forever, but forever's come and gone",
  "And now I'm left to face this empty dawn",
  "Your smile, a distant memory, a fleeting thought",
  "I'm searching for a way to heal my heart, to make it stop"],
 "pre_chorus": ["But the pain remains, like an open sore",
  "A constant reminder of what we had before",
  "I'm trying to move on, but I'm stuck in this place",
  "Where love and loss collide, and my heart can't escape"],
 "chorus": ["And I'm fading away, like a ghost in the night",
  "My heart is breaking, but it's not feeling right",
  "I'm trying to hold on, but you're slipping through my hands",
  "Leaving me with nothing but these tears and this pain"],
 "verse_two": ["We said forever, but forever's just a lie",
  "And now I'm left to wonder why",
  "You walked away, without a fight",
  "Leaving me here, to face the dark of night"],
 "outro": ["And I'll keep on fading, into the shadows of my mind",
  "Where love and loss entwine, and my heart will forever be left behind",
  "But maybe someday, I'll learn to let go",
  "And find my way back home, where love will set me free"]}


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from typing import List, Dict
from smolagents import tool
import textstat

@tool
def count_syllables(stanza: List[str]) -> List[Dict[str, int]]:
    """
    Counts the number of syllables in each verse of a stanza.

    Args:
        stanza: A list of verse lines.
    
    Returns:
        A list of dictionaries with the verse text and syllable count.
    """
    return [{"text": line.strip(), "syllables": textstat.syllable_count(line.strip())} for line in stanza]


@tool
def detect_rhyme_scheme(stanza: List[str]) -> List[str]:
    """
    Detects a basic rhyme scheme by comparing the last two letters of each verse.

    Args:
        stanza: A list of verse lines.
    
    Returns:
        A list representing the rhyme labels (A, B, C, ...).
    """
    suffixes = []
    rhyme_map = {}
    label = "A"
    
    for line in stanza:
        end = line.strip()[-2:].lower() if len(line.strip()) >= 2 else ""
        if end not in rhyme_map:
            rhyme_map[end] = label
            label = chr(ord(label) + 1)
        suffixes.append(rhyme_map[end])
    
    return suffixes


# @tool
# def detect_emotion(stanza: List[str]) -> str:
#     """
#     Roughly estimates the dominant emotion in the stanza using keywords.

#     Args:
#         stanza: A list of verse lines.
    
#     Returns:
#         A string representing a simple emotion label.
#     """
#     text = " ".join(stanza).lower()
#     if any(word in text for word in ["love", "happy", "shine", "sun"]):
#         return "joy"
#     elif any(word in text for word in ["cry", "dark", "alone", "pain"]):
#         return "sadness"
#     elif any(word in text for word in ["fight", "burn", "rage", "war"]):
#         return "anger"
#     else:
#         return "neutral"


@tool
def extract_keywords(stanza: List[str], top_k: int = 5) -> List[str]:
    """
    Extracts the most frequent keywords from the stanza.

    Args:
        stanza: A list of verse lines.
        top_k: Number of keywords to extract.

    Returns:
        A list of keywords.
    """
    text = " ".join(stanza)
    vectorizer = CountVectorizer(stop_words="english", max_features=top_k)
    X = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out().tolist()

In [14]:
from smolagents import CodeAgent

agent = CodeAgent(
    tools=[analyze_verse_metrics, detect_rhyme_scheme, extract_keywords],
    model=smolagents_model
)

answer = agent.run(
    """
    Consider the lyrics of the following song: {song}
    First, count the syllables in each verse to determine the ideal compass for the song.
    Then, extract the keywords and determine the emotion of each stanza, in order to plan the emotion of the harmony to accompany it.
    Your final answer will be a report that will be sent to the agents who will create the rythm and harmony for this song.
    Use only the tool you are provided with. 
    """.format(song=str(lyrics))
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Consider the lyrics of the following song: {'title': 'Fading Away', 'intro': ['In the silence, I hear your      │
│ voice', 'A whispered reminder of our choice', 'To let go, to move on, to lose', 'The love we had, the memories  │
│ we chose'\], 'verse_one': ["We said forever, but forever's come and gone", "And now I'm left to face this empty │
│ dawn", 'Your smile, a distant memory, a fleeting thought', "I'm searching for a way to heal my heart, to make   │
│ it stop"\], 'pre_chorus': ['But the pain remains, like an open sore', 'A constant reminder of what we had       │
│ before', "I'm trying to move on, but I'm stuck in this place", "Where love and loss collide, and my heart can't │
│ escape"\], 'chorus': ["And I'm fading away, like a ghost in the night", "My heart is breaking, but it's not     │
│ feeling right", "I'm trying to hold on, but you're slipping through my hands", 'Leaving me with nothing but     │
│ these tears and this pain'\], 'verse_two': ["We said forever, but forever's just a lie", "And now I'm left to   │
│ wonder why", 'You walked away, without a fight', 'Leaving me here, to face the dark of night'\], 'outro': ["And │
│ I'll keep on fading, into the shadows of my mind", 'Where love and loss entwine, and my heart will forever be   │
│ left behind', "But maybe someday, I'll learn to let go", 'And find my way back home, where love will set me     │
│ free'\]}                                                                                                        │
│     First, count the syllables in each verse to determine the ideal compass for the song.                       │
│     Then, extract the keywords and determine the emotion of each stanza, in order to plan the emotion of the    │
│ harmony to accompany it.                                                                                        │
│     Your final answer will be a report that will be sent to the agents who will create the rythm and harmony    │
│ for this song.                                                                                                  │
│     Use only the tool you are provided with.                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  lyrics = {                                                                                                       
      'title': 'Fading Away',                                                                                      
      'intro': ['In the silence, I hear your voice', 'A whispered reminder of our choice', 'To let go, to move     
  on, to lose', 'The love we had, the memories we chose'],                                                         
      # ... rest of the song structure                                                                             
  }                                                                                                                
                                                                                                                   
  for section_name in lyrics:                                                                                      
      if isinstance(lyrics[section_name], list):                                                                   
          for line in lyrics[section_name]:                                                                        
              print(f"Verse metrics {section_name}:")                                                              
              verse_metrics = analyze_verse_metrics(line)                                                          
              print("Metrics:", verse_metrics)                                                                     
                                                                                                                   
  # Output sample from analyzing the 'intro' section                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Verse metrics intro:
Metrics: [{'text': 'I', 'syllables': 1}, {'text': 'n', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 't',
'syllables': 1}, {'text': 'h', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 's', 'syllables': 1}, {'text': 'i', 'syllables': 1}, {'text': 'l', 'syllables': 1}, {'text': 'e', 
'syllables': 1}, {'text': 'n', 'syllables': 1}, {'text': 'c', 'syllables': 1}, {'text': 'e', 'syllables': 1}, 
{'text': ',', 'syllables': 0}, {'text': '', 'syllables': 0}, {'text': 'I', 'syllables': 1}, {'text': '', 
'syllables': 0}, {'text': 'h', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': 'a', 'syllables': 1}, 
{'text': 'r', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'y', 'syllables': 1}, {'text': 'o', 
'syllables': 1}, {'text': 'u', 'syllables': 1}, {'text': 'r', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'v', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': 'i', 'syllables': 1}, {'text': 'c', 
'syllables': 1}, {'text': 'e', 'syllables': 1}]
Verse metrics intro:
Metrics: [{'text': 'A', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'w', 'syllables': 3}, {'text': 'h',
'syllables': 1}, {'text': 'i', 'syllables': 1}, {'text': 's', 'syllables': 1}, {'text': 'p', 'syllables': 1}, 
{'text': 'e', 'syllables': 1}, {'text': 'r', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': 'd', 
'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'r', 'syllables': 1}, {'text': 'e', 'syllables': 1}, 
{'text': 'm', 'syllables': 1}, {'text': 'i', 'syllables': 1}, {'text': 'n', 'syllables': 1}, {'text': 'd', 
'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': 'r', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'o', 'syllables': 1}, {'text': 'f', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'o', 
'syllables': 1}, {'text': 'u', 'syllables': 1}, {'text': 'r', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'c', 'syllables': 1}, {'text': 'h', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': 'i', 
'syllables': 1}, {'text': 'c', 'syllables': 1}, {'text': 'e', 'syllables': 1}]
Verse metrics intro:
Metrics: [{'text': 'T', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'l',
'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': 't', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'g', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': ',', 'syllables': 0}, {'text': '', 
'syllables': 0}, {'text': 't', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'm', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': 'v', 'syllables': 1}, {'text': 'e', 
'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'o', 'syllables': 1}, {'text': 'n', 'syllables': 1}, 
{'text': ',', 'syllables': 0}, {'text': '', 'syllables': 0}, {'text': 't', 'syllables': 1}, {'text': 'o', 
'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'l', 'syllables': 1}, {'text': 'o', 'syllables': 1}, 
{'text': 's', 'syllables': 1}, {'text': 'e', 'syllables': 1}]
Verse metrics intro:
Metrics: [{'text': 'T', 'syllables': 1}, {'text': 'h', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': '',
'syllables': 0}, {'text': 'l', 'syllables': 1}, {'text': 'o', 'syllables': 1}, {'text': 'v', 'syllables': 1}, 
{'text': 'e', 'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'w', 'syllables': 3}, {'text': 'e', 
'syllables': 1}, {'text': '', 'syllables': 0}, {'text': 'h', 'syllables': 1}, {'text': 'a', 'syllables': 1}, 
{'text': 'd', 'syllables': 1}, {'text': ',', 'syllables': 0}, {'text': '', 'syllables': 0}, {'text': 't', 
'syllables': 1}, {'text': 'h', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': '', 'syllables': 0}, 
{'text': 'm', 'syllables': 1}, {'text': 'e', 'syllables': 1}, {'text': 'm', 'syllables': 1}, {'text': 'o', 
'syllables': 1}, {'text': 'r', 'syllables': 1}, {'text': 'i', 'syllables': 1}, {'text': '

[Step 1: Duration 201.15 seconds| Input tokens: 2,557 | Output tokens: 175]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.APIConnectionError: Ollama_chatException - Server disconnected without sending a response.

[Step 3: Duration 111.17 seconds| Input tokens: 13,187 | Output tokens: 327]

AgentGenerationError: Error in generating model output:
litellm.APIConnectionError: Ollama_chatException - Server disconnected without sending a response.

In [ ]:
from music21 import stream, note, meter, key, stream, chord
from smolagents.tools import tool
import uuid

scores = {}

@tool
def create_score(key_signature: str = "C", time_signature: str = "4/4") -> str:
    """
    Creates a new score with specified key and time signature. Returns the score ID.

    Args:
        key_signature: key signature for the score, like C, D, E.
        time_signature: time signature for the score, like 4/4, 3/4, 2/4.
    """
    s = stream.Score()
    p = stream.Part()
    ks = key.Key(key_signature)
    ts = meter.TimeSignature(time_signature)
    p.append(ks)
    p.append(ts)
    s.append(p)

    score_id = str(uuid.uuid4())
    scores[score_id] = {"score": s, "part": p}
    print(f"New score created with id {score_id}")

    return score_id

@tool
def add_chord_to_score(score_id: str, pitches: list[str], duration: str = "quarter") -> None:
    """
    Adds a chord to the specified score.

    Args:
        score_id: ID of the score
        pitches: list of pitches, e.g., ["C4", "E4", "G4"]
        duration: duration of the chord ("whole", "half", "quarter", "eighth", "16th", etc.)
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    c = chord.Chord(pitches)
    c.duration.type = duration
    scores[score_id]["part"].append(c)
    print(f"Added chord {pitches} ({duration}) to score {score_id}.")

@tool
def add_lyrics_to_score(score_id: str, lyrics: list[str]) -> str:
    """
    Adds lyrics to the notes of the specified score.

    Args:
        score_id: ID of the score
        lyrics: list of lyrics (e.g., ["Hel-", "lo", "world"])

    Notes:
        Lyrics are assigned sequentially to each note or chord in the part.
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    part = scores[score_id]["part"]
    notes_and_chords = [n for n in part.recurse().notes]  # includes both notes and chords
    
    if len(lyrics) > len(notes_and_chords):
        return f"Error: More lyrics than notes/chords ({len(lyrics)} > {len(notes_and_chords)})."
    
    for i, lyric in enumerate(lyrics):
        notes_and_chords[i].lyric = lyric

    return f"Added {len(lyrics)} lyrics to score {score_id}."


@tool
def add_note_to_score(score_id: str, pitch: str, duration: str = "quarter") -> None:
    """
    Adds a note to the specified score.

    Args:
        score_id: id of the score
        pitch: pitch of the note, like "C4", "D5", "B3"
        duration: duration of the note ("whole", "half", "quarter", "eighth", "16th", "32nd", "64th")
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    n = note.Note(pitch)
    n.duration.type = duration
    scores[score_id]["part"].append(n)
    print(f"Added {pitch} ({duration}) to score {score_id}.")


def get_lyrics() -> str:
    """
    Gets the lyrics of the song.
    """
    sections = [
        'intro', 'verse_one', 'pre_chorus', 'chorus', 'verse_two', 'pre_chorus', 'chorus', 'outro'
    ]

    full_lyrics = ''

    for section in sections:
        full_lyrics += '(' + section + ')\n'
        for l in lyrics[section]:
            full_lyrics += l
            full_lyrics += '\n'
        full_lyrics += '\n'

    return full_lyrics


def export_score(score_id: str, format: str = "musicxml") -> None:
    """
    Exports the score to a file and returns the file path.
    """
    if score_id not in scores:
        return f"Error: score ID {score_id} not found."

    path = f"{score_id}.{format}"
    scores[score_id]["score"].write(fmt=format, fp=path)
    print(f"Score exported to: {path}")


agent = CodeAgent(
    tools=[create_score, add_note_to_score, add_chord_to_score],
    model=smolagents_model
)

score_id = agent.run('''
    You are an experienced composer. Consider a song with the following requisites:
    - Style: {style}
    - Prompt: {user_prompt}
    - Tone: {tone}
    - Lyrics: {lyrics}
    First, create a score in C major and in a time signature of your choice.
    Then, create the chords of the song according to the lyrics.
'''.format(
        style=dict_prompt['style'],
        user_prompt=dict_prompt['user_prompt'],
        tone=dict_prompt['tone'],
        lyrics=get_lyrics()
    )
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an experienced composer. Consider a song with the following requisites:                                 │
│     - Style: Pop                                                                                                │
│     - Prompt: A song about love and resilience                                                                  │
│     - Tone: Sad                                                                                                 │
│     - Chorus: ["And I'm fading away, like a ghost in the night", "My heart is breaking, but it's not feeling    │
│ right", "I'm trying to hold on, but you're slipping through my hands", 'Leaving me with nothing but these tears │
│ and this pain'\]                                                                                                │
│     First, create a score in C major and in a time signature of your choice.                                    │
│     Then, create a simple but captvating melody for the chorus of the song, according to the lyrics.            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  score_id = create_score('C', '4/4')                                                                              
  print(f"Score ID created: {score_id}")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

New score created with id 0bdf0edd-bcfd-4cea-bf90-382066f337cf


Execution logs:
Score ID created: 0bdf0edd-bcfd-4cea-bf90-382066f337cf

Out: None

...

[Step 5: Duration 150.54 seconds| Input tokens: 14,638 | Output tokens: 830]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(score_id)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 0bdf0edd-bcfd-4cea-bf90-382066f337cf

[Step 6: Duration 44.80 seconds| Input tokens: 18,938 | Output tokens: 880]

In [ ]:
export_score('0bdf0edd-bcfd-4cea-bf90-382066f337cf')

Score exported to: 0bdf0edd-bcfd-4cea-bf90-382066f337cf.musicxml


: 